<a href="https://colab.research.google.com/github/adklinge1/QueerGan/blob/main/Queer_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_3_style_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This Queer Does Not Exists 

AI reflects the perspectives of those who design them - with  hetronormative, binary conceptions of gender. According to a [research](https://www.colorado.edu/today/2019/10/08/facial-recognition-software-has-gender-problem) by the University of Colorado Boulder, the most popular facial recognition technologies in the market are guilty of inaccurately classifying the gender of trans and non-gender-conforming individuals. Nonetheless, industries and governments around the world quickly adopt cis-sexist AI applications (such as Gender detection algorithms), and thus perpetuate the system of oppression and relegates those who do not fit into gender binaries to a subclass of human existence.

StyleGan2, like Gender Detection AI, also preserves the bias by generating only images of heteronormative women and men. This bias is likely inherited from the dataset StyleGan was trained on.
___



This project is a radical version of the [NVidia StyleGAN2 ADA](https://github.com/NVlabs/stylegan2-ada) project, which aims to generate **only Queer people**.  
 
["Queer"](https://en.wikipedia.org/wiki/Queer) is an umbrella term for gender minorities who are not [heterosexual](https://en.wikipedia.org/wiki/Heterosexuality) or are not [cisgender](https://en.wikipedia.org/wiki/Cisgender). 




 ___

In this project we used different tools :
* NVidia StyleGAN2 ADA](https://github.com/NVlabs/stylegan2-ada) model
* 

Like Audre Lord, one of the most famous femenist activists, claimed : "“The master's tools will never dismantle the master's house” 



In [ ]:
# Mount G-Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Next, clone StyleGAN2 ADA PyTorch and Gender-And-Age-Detection from GitHub.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

# !git clone https://github.com/smahesh29/Gender-and-Age-Detection.git
!pip install opencv-python
!pip install cvlib
!pip install argparse


## Run StyleGan2 From Command Line
The code below is based on code from NVidia. This actually generates your images. When you use StyleGAN you will generally create a GAN from a seed number, such as 6600.  GANs are actually created by a latent vector, containing 512 floating point values.  The seed is used by the GAN code to generate these 512 values.  The seed value is easier to represent in code than a 512 value vector.  However, while a small change to the latent vector results in a small change to the image, even a small change to the seed value will produce a radically different image.



In [ ]:
!python /content/stylegan2-ada-pytorch/generate.py \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl \
  --outdir=/content/results --seeds=8193

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl"...
Generating image for seed 8193 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


Next, copy the images to a folder of your choice on GDrive.

## Run StyleGAN2 From Python Code

Add the StyleGAN2 folder to Python so that you can import it.  The code below is based on code from NVIDIA. This actually generates your images.

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2-ada-pytorch")
import pickle
import os
import numpy as np
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt
import IPython.display
import torch
import dnnlib
import legacy

def seed2vec(G, seed):
  return np.random.RandomState(seed).randn(1, G.z_dim)

def display_image(image):
  plt.axis('off')
  plt.imshow(image)
  plt.show()

def generate_image(G, z, truncation_psi):
    # Render images for dlatents initialized from random seeds.
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    label = np.zeros([1] + G.input_shapes[1][1:])
    images = G.run(z, label, **G_kwargs) # [minibatch, height, width, channel]
    return images[0]

def get_label(G, device, class_idx):
  label = torch.zeros([1, G.c_dim], device=device)
  if G.c_dim != 0:
      if class_idx is None:
          ctx.fail('Must specify class label with --class when using a conditional network')
      label[:, class_idx] = 1
  else:
      if class_idx is not None:
          print ('warn: --class=lbl ignored when running on an unconditional network')
  return label

def generate_image(device, G, z, truncation_psi=1.0, noise_mode='const', class_idx=None):
  z = torch.from_numpy(z).to(device)
  label = get_label(G, device, class_idx)
  img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  return PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')

Load the model 

In [ ]:
URL = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"

print(f'Loading networks from "{URL}"...')
device = torch.device('cuda')
with dnnlib.util.open_url(URL) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...


In [ ]:
import sys
# sys.path.insert(0, "/Gender-and-Age-Detection")
import cv2
import cvlib as cv
import sys
import numpy as np
from termcolor import colored

def Generate_queer_img(G, seed, STEPS = 100, outputDir = 'queers'):
  os.makedirs(f"./results/{outputDir}", exist_ok=True)

  v1 = seed2vec(G, seed)
  initial_seed_img_path = f'/content/results/seed-{seed}.png';
  img = generate_image(device, G, v1)
  img.save(initial_seed_img_path)

  # detect gender
  isWoman, confidence = Is_Woman(initial_seed_img_path)
  print(colored(f'Seed {seed} was identified as {("woman" if isWoman else "man")}. (Confidence : {confidence if isWoman else 100-confidence})', 'green'))
  
  # Generate partner of opposite sex
  partnerSeed, pConfidence = Generate_Seed_By_Gender(G, False) if isWoman else Generate_Seed_By_Gender(G, True)
  print(colored(f'partner seed {partnerSeed} was identified as {("man" if isWoman else "woman")}. (Confidence : {pConfidence}', 'green'))

  parnerVector = seed2vec(G, partnerSeed)
  
  # Breed to get the queer child 
  diff = parnerVector - v1;
  stepSize = diff / STEPS

  # Move in latent space towards the partner untill the gender is flipped
  for i in range(1, STEPS):
    child = v1 + (i * stepSize)
    childPath = f'/content/results/temp/seed_{seed}+{i}_steps_to_seed_{partnerSeed}.png'
    img = generate_image(device, G, child)
    img.save(childPath)
    childIsWoman, confidence = Is_Woman(childPath)
    genderFlipped = childIsWoman != isWoman
    if genderFlipped : 
      outdirPath = f'/content/results/{outputDir}/seed_{seed}+{i}_steps_to_seed_{partnerSeed}_{confidence}%_woman.png'
      img.save(outdirPath)
      print(colored(f'Queer child of seeds: {seed} + {partnerSeed} was saved in : {outdirPath}', 'blue'))

      break;

def Is_Woman(img_path, required_confidence = 50):
  img = cv2.imread(img_path)
  label, confidence = cv.detect_gender(img)
  # print(colored(f'path : {img_path}. Confidence : {confidence[1]*100} % woman', 'yellow'))
  return (confidence[1] * 100) > required_confidence, (confidence[1] * 100)

def Generate_Seed_By_Gender(G, woman, TRIALS = 10):
  for i in range(0, TRIALS):
    randSeed = Get_Random_Seed()
    v = seed2vec(G, randSeed)
    img = generate_image(device, G,v)
    randSeedPath = f'/content/results/Seed-{randSeed}.png'
    img.save(randSeedPath)
    isRandSeedWoman, confidence = Is_Woman(randSeedPath)

    if(woman == isRandSeedWoman):

      # print(colored(f'Seed {randSeed} was identified as a {("woman" if isRandSeedWoman else "man")}' ,'green'))
      return randSeed, (confidence if woman else 100 - confidence)
    
    # delete img
    !rm '{randSeedPath}'

def Get_Random_Seed(maxValue = 100000):
  rng = np.random.default_rng()
  rints = rng.integers(low=0, high=maxValue, size=1)
  # print(f'Random seed : {rints[0]}')
  return rints[0]


Run the bellow snippet to Generate the desired amount of Queer images

In [ ]:
import shutil

NumberOfQueerPhotos = 10;

for i in range(0, NumberOfQueerPhotos):
  randomSeed = Get_Random_Seed()

  # clear temportal results of previous breeding processes
  shutil.rmtree('./results/temp', ignore_errors=True)


  os.makedirs("./results/temp", exist_ok=True)
  Generate_queer_img(G, randomSeed)




---

# Copy the Queers folder into you drive 


In [ ]:
cp /content/results/queers/* \
    /content/drive/MyDrive/QueerGan